# Bed Reflectivity Calculation

Now, take the raw bed-echo power, calculated in ./get-bed-power.ipynb, and use that to calculate a more 'physical' field for the bed reflectivity. This includes levelling the power at crossover points and correcting for spreading and attenuation losses.

We also sort the traces by location into 'grounded', 'ungrounded', and by the known subglacial lakes

In [ ]:
### Standard Imports ###

import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
### Get the bed pick dataframe ###

# If the previous notebook has not been run to get bed-echo power download it from Zenodo
fn = './Picked_Bed_Power.csv'
if not os.path.exists(fn):
    os.system('wget -L https://zenodo.org/records/11201199/files/Picked_Bed_Power.csv')
# Read as pandas dataframe
df = pd.read_csv('./Picked_Bed_Power.csv')

In [ ]:
### Filter the dataset to Whillans Ice Plain only ###

# All flight days
fdays = np.sort(df['Flight Day'].unique())

# Kamb flight days
kamb_days = fdays[[1,3,4,7]]
# Whillans flight days
whillans_days = np.delete(fdays,[1,3,4,7])

# Separate dataframes
kamb_df = df[df['Flight Day'].isin(kamb_days)]
df = df[df['Flight Day'].isin(whillans_days)]
# Add in a single frame from a kamb day where part of the flight was over SLW
df = pd.concat([df, kamb_df[(kamb_df['Flight Day']==20131227) & (kamb_df['Segment'] == 12)]])
whillans_days = np.append(whillans_days,20131227)

# Subset Whillans dataframe by y coordinate
df = df[df['Y'] > -1e6]
df = df[df['Y'] < -.48e6]

# Remove traces with large aircraft pitch or roll
df = df[abs(df['Roll']) < 0.1]
df = df[abs(df['Pitch']) < 0.075]

# Empty array for adjusted bed power
df['P_bed_adj'] = df['P_bed'].copy()

In [ ]:
### A function to level the power at crossover points ###

def crossover_leveling(df,cal_variable,days,days_to_add,crossover_dist=100,print_num=5000,verbose=False):
    
    # Separate into two dataframes, 1st the 'true' power and 2nd to adjust
    idxs1 = df['Flight Day'].isin(days)
    idxs2 = df['Flight Day'].isin(days_to_add)
    df1 = df.loc[idxs1]
    df2 = df.loc[idxs2]
    
    # Empty array for power differences at crossovers
    dP = np.array([])
    
    # Loop through every trace in the 2nd dataframe
    for i,idx in enumerate(df2.index):
        if i%print_num==0 and verbose:
            print(round(i/len(df2),3)*100,'% finished...')
            print('Total crossover traces added:',len(dP))
        # Get the distance from the selected trace to every trace in df1
        dist = np.sqrt((df2.loc[idx,'X']-df1['X'])**2.+(df2.loc[idx,'Y']-df1['Y'])**2.)
        # Add the power difference for 
        dP = np.append(dP,df1[cal_variable][dist<crossover_dist] - df2.loc[idx,cal_variable])

    # Adjust the power for the added days
    df.loc[idxs2,'P_bed_adj'] = df.loc[idxs2,'P_bed'] + np.nanmean(dP)

    if verbose:
        print("Power adjusted by:",np.nanmean(dP),' for days:',days_to_add)
        
    return df, dP

In [ ]:
### Do the crossover power adjustment for each day ###

crossover_variable = 'P_surf'

# Initialize the figures and colors to plot for each day
plt.figure()
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

for i in range(1,9):
    # Do the adjustment for each day, using the function from the previous cell
    print("Crossover Leveling Adjustment #",i)
    print("Control Days:",np.delete(whillans_days,i))
    print("Adjusted Days:",whillans_days[i])
    df, dP = crossover_leveling(df,crossover_variable,np.delete(whillans_days,i),[whillans_days[i]],crossover_dist=50,print_num=10000,verbose=True)
    print("\n")
    
    # Plot a histogram of power difference at crossover points
    plt.hist(-dP,bins=len(dP)//100,color=colors[i],alpha=0.25,label=whillans_days[i])
    plt.axvline(-np.nanmean(dP),color=colors[i])

# Save a variable for the adjusted surface power
df['P_surf_adj'] = df['P_surf'] + (df['P_bed_adj'] - df['P_bed'])

plt.ylim(0,1000)
plt.xlabel('Crossover Power (dB)')
plt.ylabel('Number of Traces')
plt.legend(title='Flight Day',fontsize=8)

In [ ]:
### Plot the adjusted power by day ###

plt.figure(figsize=(10,4))

# Map view plot
ax1 = plt.subplot(121)
for i in range(9):
    df[df['Flight Day']==whillans_days[i]].plot.scatter(ax=ax1,x='X',y='Y',c=colors[i],s=1,label=whillans_days[i])

# Histogram
ax2 = plt.subplot(122)
bins = np.arange(-160,-70,0.1)
plt.hist(df['P_bed_adj'],bins=bins,color='lightgrey');
for i,day in enumerate(whillans_days):
    plt.hist(df[df['Flight Day']==day]['P_bed_adj'],bins=bins,color=colors[i],label=day)
plt.legend(title='Flight Day',fontsize=8)

In [ ]:
### Calculate a relative reflectivity ###

# ice permittivity
epsr_ice = 3.15
# geometric spreading losses (spherical)
G = 20*np.log10(2.*(df['h'] + df['H']/np.sqrt(epsr_ice)))

# Attenuation correction
N = 10                   # one-way rate (dB/km)
L = 2.*N * df['H']/1000. # total attenuative losses

# Reflectivity
df['Relative Reflectivity'] = df['P_bed_adj'] + G + L
# Move the mean to 0
df['Relative Reflectivity'] -= df['Relative Reflectivity'].mean()

### Plot the output reflectivity field ###

plt.figure(figsize=(6,5))

ax1 = plt.subplot(111)
df.plot.scatter(ax=ax1,x='X',y='Y',c='Relative Reflectivity',cmap='magma',s=1,vmin=-25,vmax=15)
ax1.axis('equal')
plt.xlim(-300000,-100000)
plt.ylim(-800000,-500000)
plt.tight_layout()

In [ ]:
### Import Scripps grounding line and create a mask for traces grounded/floating ###

import json
from osgeo import ogr
import matplotlib.path as mplPath
import geopandas as gpd

# a base directory where I keep data to be loaded below
data_dir = os.getenv('DATAHOME')
# quick error check to make sure $DATAHOME is set
if data_dir is None:
    raise OSError('environment variable $DATAHOME does not exist')

# Scripps Grounding Line
gl = data_dir + 'continental-data-products/antarctica/grounding-line/scripps/scripps_antarctica_polygons_v1.shp' 
gl_df = gpd.read_file(gl)
gl_coords = np.transpose(gl_df.iloc[1010].geometry.exterior.coords.xy)

# Crop to the Whillans Ice Plain region
start = 46000
end = start+3000
gl_crop = gl_coords[start:end]
gl_crop = np.append(gl_crop,np.array([[-1000000,0],[1000000,0],gl_crop[0]]),axis=0)

# Label the radar points as grounded or not
gl_Path = mplPath.Path(np.transpose([gl_crop[:,0],gl_crop[:,1]]))
df['Grounded'] = gl_Path.contains_points(df[['X','Y']]).astype(bool)

In [ ]:
### Import lake outlines and create a mask for traces lake/not-lake ###

import h5py

# Subglacial lake outlines (Siegfried & Fricker, 2018)
lakes = h5py.File('../data/outlines/SiegfriedFricker2018-outlines.h5', 'r')
wh_lake_keys = ['EngelhardtSubglacialLake', 'WhillansSubglacialLake', 'Whillans_6', 'Whillans_7', 'Whillans_8','Lake78','Lake10']

# empty column to be filled
df['Lake'] = np.zeros_like(df['X']).astype(bool)
# loop through all lake keys
for key in wh_lake_keys:
    lake_x = np.squeeze(lakes[key]['x'][:])
    lake_y = np.squeeze(lakes[key]['y'][:])
    lake_Path = mplPath.Path(np.transpose([lake_x,lake_y]))
    df[key] = lake_Path.contains_points(df[['X','Y']]).astype(bool)
    df['Lake'] = np.logical_or(df['Lake'],df[key])

In [ ]:
### Plot the relative reflectivity from each of the spatially sorted fields ###

plt.figure(figsize=(12,3))

# Grounded
ax1 = plt.subplot(131)
df[df['Grounded']==True].plot.scatter(ax=ax1,x='X',y='Y',c='Relative Reflectivity',cmap='magma',s=1,vmin=-25,vmax=15)
plt.plot(gl_crop[:,0],gl_crop[:,1],'k-')
ax1.axis('equal')
plt.xlim(-300000,-100000)
plt.ylim(-800000,-500000)

# Subglacial Lake
ax2 = plt.subplot(132)
df[df['Lake']==True].plot.scatter(ax=ax2,x='X',y='Y',c='Relative Reflectivity',cmap='magma',s=1,vmin=-25,vmax=15)
plt.plot(gl_crop[:,0],gl_crop[:,1],'k-')
ax2.axis('equal')
plt.xlim(-300000,-100000)
plt.ylim(-800000,-500000)

# Not Grounded (Ice Shelf)
ax3 = plt.subplot(133)
df[df['Grounded']==False].plot.scatter(ax=ax3,x='X',y='Y',c='Relative Reflectivity',cmap='magma',s=1,vmin=-25,vmax=15)
plt.plot(gl_crop[:,0],gl_crop[:,1],'k-')
ax3.axis('equal')
plt.xlim(-300000,-100000)
plt.ylim(-800000,-500000)

# Plot lake outlines
for key in lakes.keys():
    lake_x = np.squeeze(lakes[key]['x'][:])
    lake_y = np.squeeze(lakes[key]['y'][:])
    for ax in [ax1,ax3]:
        ax.plot(lake_x,lake_y,c='steelblue')
        
plt.tight_layout()

In [ ]:
### Save the calculated reflectivity dataframe as a new csv file ###

df.to_csv('./Processed_Reflectivity.csv')